### Agents in LangChain are components that utilize Language Models along with a sequence of actions to take. The agent uses a Reasoning Engine to determing which actions to take to get a result. Agents are crucial for handling tasks ranging from simple automated responses to complex, context-aware interactions

### Tools are interfaces that an agent can use to interact with the world. Tools typically comprise of the following:
1. Name of the tool
2. A description of what the tool does
3. JSON Schema of what the input to the tool are
4. The function to call
5. Whether the result of a tool should be returned directly to the user

### Importantly, the name, description and JSON schema, are all used in the prompt (if used). Therefore, it is really important that they are clear and describe exactly how the tool should be used. We may need to change the default name, description or JSON schema if the LLM is not understanding how to use the tool.

### Agent identifies key ideas in users questions, looks at all the tools we have put at its disposal (specifically the tools title and descriptions), and then combined with the system prompt and the optional agent instructions, decides on which tool to use

### ReAct Agents
#### ReAct, stands for Reasoning + Acting, combines reasoning and acting to solve complex language reasoning and decision-making tasks. This is achieved through text actions that the model can use to ask questions or perform tasks to gain more information and better understand a situation. For instance, when faced with a multi-hop reasoning question, ReAct might initiate mnultiple search actions, or calls to external tools or custom tools within your program.

### Agents and Tools

### Agents: Agents enable LLMs to work with tools that perform various tasks
To use agents, we require three things:
1. A base LLM
2. A tool that will be interacting with
3. An agent to control the interaction

### In the following example, we will use the Math tool to perform some simple math operations

In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAI
import warnings
warnings.filterwarnings("ignore")
load_dotenv()

True

In [2]:
llm = OpenAI(
    temperature = 0,
    model = "gpt-3.5-turbo-instruct"
)
# Put temperature = 0 here because we do not want creativity with Math, we want determined mathematical reasoning

In [5]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

In [11]:
llm_math = LLMMathChain(llm=llm)

math_tool = Tool(
    name = "Calculator",
    func = llm_math.run,
    description = "Useful for when you need to answer questions about math."
)

tools = [math_tool]

In [13]:
print(tools[0].name, "--->", tools[0].description)

Calculator ---> Useful for when you need to answer questions about math.


### Creating an agent

#### Zero-shot ReAct
The zero shot ReAct Agent is a language generation model that can create realistic contexts even without being trained on specific data. It can be used for various tasks such as generating creative text formats, language translation, and generating different types of creative content.

In [18]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 10
)

In [30]:
zero_shot_agent("What is the root over of 25")



> Entering new AgentExecutor chain...
 I should use a calculator to find the square root
Action: Calculator
Action Input: 25
Observation: Answer: 25
Thought: This doesn't seem right, I should double check
Action: Calculator
Action Input: 25
Observation: Answer: 25
Thought: I should try using a different function
Action: Calculator
Action Input: 25
Observation: Answer: 25
Thought: I should try using the square root function
Action: Calculator
Action Input: 25
Observation: Answer: 25
Thought: I now know the final answer
Final Answer: 5

> Finished chain.


{'input': 'What is the root over of 25', 'output': '5'}

In [28]:
zero_shot_agent("What is the root over of 25 and 100")



> Entering new AgentExecutor chain...
 I should use a calculator to find the square root
Action: Calculator
Action Input: 25, 100
Observation: Answer: 125
Thought: I should take the square root of 125
Action: Calculator
Action Input: 125
Observation: Answer: 125
Thought: I now know the final answer
Final Answer: The root over of 25 and 100 is 125.

> Finished chain.


{'input': 'What is the root over of 25 and 100',
 'output': 'The root over of 25 and 100 is 125.'}

#### Look at this square root of 100 and 25 is 125, which is not right. This can be further improved with few shot learning agents. One more reason for me to believe GenAI might be a lot hyped up when it comes to logical reasoning. Although, it does make life easy. :-p

In [37]:
zero_shot_agent("What is the root over of 25 and 100 separately")



> Entering new AgentExecutor chain...
 I should use the Calculator tool to find the square root of a number.
Action: Calculator
Action Input: 25
Observation: Answer: 25
Thought: I should repeat the same action for 100.
Action: Calculator
Action Input: 100
Observation: Answer: 100
Thought: I now know the final answer.
Final Answer: The square root of 25 is 5 and the square root of 100 is 10.

> Finished chain.


{'input': 'What is the root over of 25 and 100 separately',
 'output': 'The square root of 25 is 5 and the square root of 100 is 10.'}

### Way of prompting | asking what we need |  matters here.

In [49]:
problem = """
    You are building a house. There are two bedrooms of 5 metres by 5 metres each 
    and drawing cum open kitchen is 7 metres by 6 metres 
    and balcony of 3 metres by 2 metres. 
    What is the total area of your house?
"""

zero_shot_agent(problem)



> Entering new AgentExecutor chain...
 To find the total area of the house, we need to calculate the area of each room and then add them together.
Action: Calculator
Action Input: 5 * 5
Observation: Answer: 25
Thought: This is the area of one bedroom.
Action: Calculator
Action Input: 25 * 2
Observation: Answer: 50
Thought: This is the area of both bedrooms.
Action: Calculator
Action Input: 7 * 6
Observation: Answer: 42
Thought: This is the area of the drawing cum open kitchen.
Action: Calculator
Action Input: 3 * 2
Observation: Answer: 6
Thought: This is the area of the balcony.
Action: Calculator
Action Input: 50 + 42 + 6
Observation: Answer: 98
Thought: This is the total area of the house.
Final Answer: The total area of the house is 98 square metres.

> Finished chain.


{'input': '\n    You are building a house. There are two bedrooms of 5 metres by 5 metres each \n    and drawing cum open kitchen is 7 metres by 6 metres \n    and balcony of 3 metres by 2 metres. \n    What is the total area of your house?\n',
 'output': 'The total area of the house is 98 square metres.'}

### Using multiple tools

In [20]:
# Defining the Tools

from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()


search_tool = Tool.from_function(
    func = search.run,
    name = "Search",
    description = "useful for when you need to search the internet for information"
)

llm_math_chain = LLMMathChain(llm=llm, verbose = True)

math_tool = Tool.from_function(
    name = "Calculator",
    func = llm_math_chain.run,
    description = "Useful for when you are asked to perform math calculations"
)


In [38]:
tools = [search_tool, math_tool]

agent = initialize_agent(
    tools,
    llm,
    agent = "zero-shot-react-description",
    verbose = True,
    max_iterations = 2
)

In [48]:
agent.run("""Get Cristiano Ronaldos age from Google and display the age and birthday""")



> Entering new AgentExecutor chain...
 I should use the Search tool to find information about Cristiano Ronaldo's age and birthday
Action: Search
Action Input: 'Cristiano Ronaldo age and birthday'
Observation: Cristiano Ronaldo (born February 5, 1985, Funchal, Madeira, Portugal) is a Portuguese football (soccer) forward who is one of the greatest players of his generation. Ronaldo is the winner of five Ballon d'Or awards (2008, 2013, 2014, 2016, and 2017), and in 2024 he became the first men's player to score 900 career goals in official matches.. Early life and career Cristiano Ronaldo Age. Ronaldo was born on 5 February 1985 and is currently 39 years old. Club career. Before joining Manchester United in 2003 and winning the FA Cup in his debut season, Ronaldo started his senior career at Sporting CP. Along with winning the FIFA Club World Cup, the Champions League, and three straight Premier League ... Cristiano Ronaldo dos Santos Aveiro was born in Funchal, Madeira Islands in earl

'Agent stopped due to iteration limit or time limit.'

### Kept it restricted to 2 iterations as it will go on for few more times to give the age and birthday using duckduckgo search

## Creating a custom tool

In [74]:
from langchain.tools import BaseTool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools import tool


@tool("lower_case", return_direct = True)
def StructuredResponseTool(question: str):
    """
    use this tool to send a prompt and get a JSON returned
    with three fields - Topic, Question_Details and Detailed_Response
    """

    json_prompt = PromptTemplate.from_template(
    """
    Return a JSON object with an 'answer' key that answers the following question: {question}.
    The JSON object will have three fields - Topic, Question_Details and Detailed_Response
    """
    )

    model = OpenAI(model_name = "gpt-3.5-turbo-instruct", temperature = 0.5)

    json_parser = JsonOutputParser()
    json_chain = json_prompt | model | json_parser
    x = json_chain.invoke({"question": question})
    return x

In [76]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [80]:
from langchain.agents import initialize_agent

tools = [StructuredResponseTool]

zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 10
)

In [90]:
zero_shot_agent.run(input="Which is the top football team in the worls and how, the response should have three fields in the JSON - topic of the question, the question and detailed response")



> Entering new AgentExecutor chain...
 I should use the lower_case tool to get the JSON response
Action: lower_case
Action Input: "Which is the top football team in the world and how"
Observation: {'Topic': 'Football', 'Question_Details': 'Which is the top football team in the world and how?', 'Detailed_Response': "The top football team in the world is currently considered to be FC Barcelona. This is based on their consistent success in major competitions such as the UEFA Champions League and La Liga, as well as their talented roster of players including Lionel Messi, Neymar, and Luis Suarez. Additionally, Barcelona's style of play, known as tiki-taka, is highly praised for its fluidity and effectiveness on the field. Overall, FC Barcelona's combination of skill, strategy, and success make them the top football team in the world."}


> Finished chain.


{'Topic': 'Football',
 'Question_Details': 'Which is the top football team in the world and how?',
 'Detailed_Response': "The top football team in the world is currently considered to be FC Barcelona. This is based on their consistent success in major competitions such as the UEFA Champions League and La Liga, as well as their talented roster of players including Lionel Messi, Neymar, and Luis Suarez. Additionally, Barcelona's style of play, known as tiki-taka, is highly praised for its fluidity and effectiveness on the field. Overall, FC Barcelona's combination of skill, strategy, and success make them the top football team in the world."}

### Definitely needs some check, FCB as the best club rn woah!!!!